# GPU-accelerated motion planning with cuRobo

In this notebook, we show you how to accelerate motion planning with [cuRobo](https://curobo.org).

Before continuing, make sure you have followed the cuRobo-specific installation instructions in the `airo-planner` README file.

## Preparing

We'll first download some necessary assets from Nvidia.

In [ ]:
! wget https://raw.githubusercontent.com/NVlabs/curobo/refs/heads/main/src/curobo/content/assets/robot/ur_description/ur5e.urdf -O /tmp/ur5e.urdf
! wget https://raw.githubusercontent.com/NVlabs/curobo/refs/heads/main/src/curobo/content/configs/robot/ur5e.yml -O /tmp/ur5e.yml
! wget https://raw.githubusercontent.com/NVlabs/curobo/refs/heads/main/src/curobo/content/configs/world/collision_table.yml -O /tmp/collision_table.yml

## Creating the environment

For creating the world, you can use a world file containing all meshes, cuboids, etcetera. Or, you can create your world programmatically. For more information, refer to [the cuRobo documentation](https://curobo.org/get_started/2c_world_collision.html).

Here, we'll use a world file.

In [ ]:
robot_file = "/tmp/ur5e.yml"
world_file = "/tmp/collision_table.yml"

In [ ]:
from airo_planner.curobo.single_arm_curobo_planner import SingleArmCuroboPlanner
from airo_planner.curobo.robot import RobotRepresentation
import numpy as np

# For visualisation, we use drake.
from pydrake.planning import RobotDiagramBuilder
from pydrake.trajectories import PiecewisePolynomial
from airo_drake import SingleArmScene, add_floor, add_manipulator, add_meshcat, finish_build, animate_joint_trajectory

robot_diagram_builder = RobotDiagramBuilder()

meshcat = add_meshcat(robot_diagram_builder)
arm_index, gripper_index = add_manipulator(robot_diagram_builder, "ur5e", "robotiq_2f_85", static_gripper=True)
add_floor(robot_diagram_builder)
robot_diagram, context = finish_build(robot_diagram_builder)

scene = SingleArmScene(robot_diagram, arm_index, gripper_index, meshcat)

## Setting up the planner

We can set up the cuRobo planner with a single line.

In [ ]:
planner = SingleArmCuroboPlanner(robot_file, world_file)

## Planning in joint state

We'll move from a start configuration to a goal configuration.

In [ ]:
q_start = np.array([ 0.0000, -2.2000,  1.9000, -1.3830, -1.5700,  0.0000])
q_goal = q_start + 0.5

In [ ]:
trajectory = planner.plan_to_joint_configuration(q_start, q_goal)

Let's visualize the trajectory in meshcat.

In [ ]:
animate_joint_trajectory(meshcat, robot_diagram, arm_index, trajectory)

## Planning to an EEF pose

To plan in EEF state, we'll first create a kinematic model with cuRobo. The `RobotRepresentation` gives us forward and inverse kinematics.

In [ ]:
robot_representation = RobotRepresentation(robot_file, "/tmp/ur5e.urdf")
tcp_pose = robot_representation.forward_kinematics(q_goal)  # robot_representation.inverse_kinematics also exists...

In [ ]:
trajectory = planner.plan_to_tcp_pose(q_start, tcp_pose)

In [ ]:
animate_joint_trajectory(meshcat, robot_diagram, arm_index, trajectory)